# mat to png (Wei)

## mat to avi

In [ ]:
import scipy.io as sio
import numpy as np
import cv2
import os


mat_contents = sio.loadmat('D:\\project\\Weizmann\\aligned_masks_dev.mat') 
aligned_masks = mat_contents['masks']
video_dir = 'D:\\project\\Weizmann\\700_700_avi_dev'
os.makedirs(video_dir, exist_ok=True)
fourcc = cv2.VideoWriter_fourcc(*'XVID')

for name in aligned_masks.dtype.names:
    mask_data = aligned_masks[name][0, 0]
    video_filename = os.path.join(video_dir, f"{name}.avi")
    out = cv2.VideoWriter(video_filename, fourcc, 30.0, (700, 700), isColor=False)

    for i in range(mask_data.shape[2]):
        frame = mask_data[:, :, i].astype(np.uint8) * 255       
        top_padding = (180 - 144) // 2
        bottom_padding = 180 - 144 - top_padding

        frame_padded = cv2.copyMakeBorder(frame, top_padding, bottom_padding, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        frame = cv2.resize(frame_padded, (700, 700))
        out.write(frame)

    out.release()

cv2.destroyAllWindows()
print("All videos have been saved in:", video_dir)



## avi to event

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import sys
sys.path.append(r"D:\\my_software\\event_camera\\Prophesee\\lib\\python3\\site-packages\\")

import numpy as np
import cv2
from metavision_core_ml.video_to_event.simulator import EventSimulator
from metavision_core_ml.data.video_stream import TimedVideoStream

def video_to_events(video_path, output_path):
    height, width = 700, 700
    stream = TimedVideoStream(video_path)

    C = 0.5  
    refractory_period = 10000  
    cutoff_hz = 30  
    sigma_threshold = 0 
    shot_noise_rate_hz = 0 

    simulator = EventSimulator(height, width, C, C, refractory_period, cutoff_hz=cutoff_hz, 
                               sigma_threshold=sigma_threshold, shot_noise_rate_hz=shot_noise_rate_hz)

    events = []
    for img, ts in stream:
        total_events = simulator.image_callback(img.squeeze(), ts)
        if total_events > 0:
            frame_events = simulator.get_events()
            events.append(frame_events)
            simulator.flush_events()  

    if events:
        events = np.concatenate(events)
        np.save(output_path, events)  

def convert_video(video_path, output_path):
    video_to_events(video_path, output_path)

def convert_videos_to_events(input_folder, output_folder):
    existing_files = os.listdir(output_folder)
    videos_to_convert = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".avi"):
            video_path = os.path.join(input_folder, filename)
            event_filename = filename.replace(".avi", ".npy")
            output_path = os.path.join(output_folder, event_filename)
            if event_filename not in existing_files:
                videos_to_convert.append((video_path, output_path))

    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_video = {executor.submit(convert_video, video_path, output_path): (video_path, output_path) for video_path, output_path in videos_to_convert}
        for future in tqdm(as_completed(future_to_video), total=len(videos_to_convert), desc="Converting videos"):
            try:
                future.result()  
            except Exception as exc:
                video_path, output_path = future_to_video[future]
                print(f'Error processing {video_path}: {exc}')

dev_input_folder = "D:\\project\\Weizmann\\700_700_avi_dev"
dev_output_folder = "D:\\project\\Weizmann\\700_700_npy_dev"


os.makedirs(dev_output_folder, exist_ok=True)
convert_videos_to_events(dev_input_folder, dev_output_folder)
# convert_videos_to_events(train_input_folder, train_output_folder)


## event to png

In [ ]:
import numpy as np
import os
from PIL import Image

folder_path = "D:\\project\\Weizmann\\700_700_npy_dev"
files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

# figure size
xx = 700
yy = 700

for file in files:
    events = np.load(os.path.join(folder_path, file))

    dtype = [('x', np.int16), ('y', np.int16), ('polarity', np.int8), ('timestamp', np.int32)]
    filtered_events = np.array([tuple(event) for event in events], dtype=dtype)

    delta_t = 1000000
    total_time = filtered_events[-1]['timestamp']
    num_frames = total_time // delta_t
    remainder = total_time % delta_t
    if remainder > (delta_t/2):
        num_frames += 1

    for i in range(num_frames):
        start_time = i * delta_t
        end_time = (i + 1) * delta_t
        time_filtered_events = filtered_events[(filtered_events['timestamp'] > start_time) & (filtered_events['timestamp'] < end_time)]

        event_counts1 = np.zeros((xx, yy), dtype=np.uint32)
        event_counts0 = np.zeros((xx, yy), dtype=np.uint32)

        positive_indices = time_filtered_events['polarity'] == 1
        negative_indices = time_filtered_events['polarity'] == 0

        np.add.at(event_counts1, (time_filtered_events[positive_indices]['y'], time_filtered_events[positive_indices]['x']), 1)

        np.add.at(event_counts0, (time_filtered_events[negative_indices]['y'], time_filtered_events[negative_indices]['x']), 1)

        event_counts_diff = event_counts1 - event_counts0
        image = np.where(event_counts_diff > 1, 255, 0).astype(np.uint8)
        image = Image.fromarray(image)

        save_path = os.path.join(folder_path.replace('700_700_npy_dev', '700_700_frame_dev'), f"{file[:-4]}_{i}.png")
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        image.save(save_path)


# raw to png (KTH)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from metavision_core.event_io import AdaptiveRateEventsIterator
from metavision_core.event_io import EventsIterator
from metavision_sdk_core import AdaptiveRateEventsSplitterAlgorithm

sys.path.append(r"C:\\Program Files\\Prophesee\\lib\\python3\\site-packages\\")
sys.path.append(os.path.dirname(r"C:\\Program Files\\Prophesee\\share\\metavision\\sdk\\core\\python_samples\\metavision_adaptive_rate"))



def events_to_diff_image(events, sensor_size, strict_coord=True):
    xs = events["x"]
    ys = events["y"]
    ps = events["p"] # ps = 0/1(-/+)

    # Filter out events outside the size of the sensor
    mask = (xs < sensor_size[1]) * (ys < sensor_size[0]) * (xs >= 0) * (ys >= 0)

    if strict_coord:
        assert (mask == 1).all()
    coords = np.stack((ys*mask, xs*mask))
    ps *= mask
   

    try:
        abs_coords = np.ravel_multi_index(coords, sensor_size)
    except ValueError:
        raise ValueError("Issue with input arrays! coords={}, min_x={}, min_y={}, max_x={}, max_y={}, coords.shape={}, sum(coords)={}, sensor_size={}".format(
            coords, min(xs), min(ys), max(xs), max(ys), coords.shape, np.sum(coords), sensor_size))

    img = np.bincount(abs_coords, weights=ps, minlength=sensor_size[0]*sensor_size[1])
    img = img.reshape(sensor_size)
    return img


# Config
delta_t = 1000000
output_dir = "D:\\project\\KTH"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

timeline_data_path = 'D:/project/dev_timeline.npy'
timeline_data = np.load(timeline_data_path, allow_pickle=True)
file_start_times = {row[0]: (float(row[1]), float(row[2])) for row in timeline_data}
first_frame = float(timeline_data[0][1])

# raw_file_path = "D:\\project\\KTH\\test_output\\train.raw"
raw_file_path = "C:\\Users\\soapy\\Documents\\metavision\\recordings\\dev.raw"

# Initialization
events_iterator = EventsIterator(raw_file_path, mode="delta_t", delta_t=delta_t)
height, width = events_iterator.get_size()


frame_indices = {name: 0 for name in file_start_times}
for frame_index, events in enumerate(events_iterator):
    current_ts = frame_index * delta_t - first_frame * 1e6  

    # Check which video segment the current event data belongs to
    for file_name, (start_t, end_t) in file_start_times.items():
        start_ts_true = (start_t - first_frame) * 1e6  
        end_ts_true = (end_t - first_frame) * 1e6 

        if start_ts_true <= current_ts <= end_ts_true:
            base_file_name = file_name.replace(".mp4", "")
            true_frame_index = frame_indices[file_name]
            image_filename = f"{base_file_name}_{true_frame_index}.png"
            img = events_to_diff_image(events, sensor_size=(height, width))
            img = np.where(img > 1, 255, 0)  # Threshold
            plt.imsave(os.path.join(output_dir, image_filename), img, cmap='gray')
            print(f"Saved {image_filename} for timeslot {file_name}")
            frame_indices[file_name] += 1  
            break


# png resize

In [ ]:
import os
from PIL import Image

src_folder = "D:/project/KTH/match_data/dev_origin"
dest_folder = "D:/project/KTH/match_data/dev"

os.makedirs(dest_folder, exist_ok=True)


for filename in os.listdir(src_folder):
    if filename.lower().endswith('.png'):  
        img_path = os.path.join(src_folder, filename)
        img = Image.open(img_path)
        
        new_width = int(img.width * 0.75)
        new_height = int(img.height * 0.75)
        
        img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS) 
        

        new_img = Image.new('RGB', (500, 500), (0, 0, 0))
        x = (500 - new_width) // 2
        y = (500 - new_height) // 2
        

        new_img.paste(img_resized, (x, y))
        new_img.save(os.path.join(dest_folder, filename))

print("finished")


# png rename and produce npy

In [ ]:
import os
from PIL import Image
import numpy as np


label_map = {'boxing': 0, 'handclapping': 1, 'handwaving': 2, 'jogging': 3, 'running': 4, 'walking': 5}

def preprocess_images(root_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    labels_txt = []  # To store numeric labels for txt file
    details_list = []  # To store the image details including person, activity, day, sequence, numeric label for npy, image counter, and concatenated label
    image_counter = 0  # To name images sequentially

    for filename in os.listdir(root_dir):
        if filename.endswith('.png'):
            parts = filename.split('_')
            if len(parts) == 5:  # Expected format: person01_boxing_d2_uncomp_10.png
                person = parts[0]
                activity = parts[1]
                day = parts[2]
                sequence_number = parts[4].split('.')[0]  # Remove the '.png'
                
                if activity in label_map:
                    # Convert string label to numeric label for txt file
                    numeric_label_txt = label_map[activity]
                    # Convert string label to numeric label for npy file
                    numeric_label_npy = label_map[activity]

                    # Construct the concatenated label
                    concatenated_label = f"{person}_{activity}_{day}_uncomp"

                    # Read and process image
                    file_path = os.path.join(root_dir, filename)
                    with Image.open(file_path) as img:
                        # Save processed image with a new sequential name
                        output_filename = f'image_{image_counter}.png'
                        output_path = os.path.join(output_dir, output_filename)
                        img.save(output_path)  # Directly save the original image under new name
                        
                        # Append details to list including the image counter and concatenated label
                        details_list.append([person, activity, day, sequence_number, numeric_label_npy, image_counter, concatenated_label])
                        
                        # Append numeric label for text file (1-6)
                        labels_txt.append(numeric_label_txt)
                        
                        # Increment image counter
                        image_counter += 1

    # Convert list to numpy array and save
    details_array = np.array(details_list, dtype='U50')  
    np.save(os.path.join(output_dir, 'bhh-jrw_dev.npy'), details_array)

    # Save labels to a text file
    with open(os.path.join(output_dir, 'labels.txt'), 'w') as f:
        for label in labels_txt:
            f.write(f"{label}\n")

# Specify the output directory and input directory
output_train_dir = 'D:/project/KTH/match_data/KTH_dev_rebal'
preprocess_images('D:/project/KTH/match_data/KTH_dev', output_train_dir)


# creat hdf5

In [ ]:
import h5py
import numpy as np
from PIL import Image


image_folder = r'D:/project/KTH/match_data/KTH_dev_rebal'
label_file = r'D:/project/KTH/match_data/KTH_dev_rebal/labels.txt'


with open(label_file, 'r') as f:
    labels = f.readlines()
labels = [int(label.strip()) for label in labels]

hdf5_file = 'D:/project/KTH/match_data/dev.hdf5'
with h5py.File(hdf5_file, 'w') as f:
    num_images = len(labels)
    image_shape = (num_images, 500, 500) 
    label_shape = (num_images,)

    images_dset = f.create_dataset('images', shape=image_shape, dtype=np.uint8)
    labels_dset = f.create_dataset('labels', shape=label_shape, dtype=np.uint8)

    for i in range(num_images):
        image_path = f'{image_folder}/image_{i}.png'
        image = Image.open(image_path).convert('L')  
        
        image_array = np.array(image)
        images_dset[i] = image_array

        labels_dset[i] = labels[i]  

print("finished")